### Data loader of pageviews of Covid-19 related articles for 175 languages.

In [3]:
import pandas as pd
import os
import gzip
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import requests
import datetime
from scipy import stats
from helper import *
import urllib.parse
import time   

In [2]:
COVID_RELATED_ARTICLES_PATH = "data/COVID_related_pages_project.csv"
df_covid_articles = pd.read_csv(COVID_RELATED_ARTICLES_PATH)
df_covid_articles.head()

,page,project,url,wikilink
0,أثر جائحة فيروس كورونا على الدين 2019-20,ar.wikipedia,https://ar.wikipedia.org/wiki/%D8%A3%D8%AB%D8%...,[[ar:أثر جائحة فيروس كورونا على الدين 2019-20
1,Impact of the COVID-19 pandemic on religion,en.wikipedia,https://en.wikipedia.org/wiki/Impact_of_the_CO...,[[en:Impact of the COVID-19 pandemic on religion
2,Impacto en la religión de la pandemia de enfer...,es.wikipedia,https://es.wikipedia.org/wiki/Impacto_en_la_re...,[[es:Impacto en la religión de la pandemia de ...
3,Dampak pandemi koronavirus terhadap kegiatan k...,id.wikipedia,https://id.wikipedia.org/wiki/Dampak_pandemi_k...,[[id:Dampak pandemi koronavirus terhadap kegia...
4,코로나바이러스감염증-19 범유행이 종교에 준 영향,ko.wikipedia,https://ko.wikipedia.org/wiki/%EC%BD%94%EB%A1%...,[[ko:코로나바이러스감염증-19 범유행이 종교에 준 영향


In [3]:
#get the 175 language code
df_lang = df_covid_articles['project'].apply(lambda p: p.split('.')[0]).drop_duplicates()
pageview_df= pd.DataFrame()
df_tmp  = pd.DataFrame()
for value in df_lang:
    #get covid related from 01-01-2020 to 31-07-2022 for each language
    df_tmp = wiki_to_df_extract(value, '20200101', '20220731', df_covid_articles).rename({'views': value}, axis='columns')
    pageview_df = pd.concat([pageview_df, df_tmp], axis= 1)

In [4]:
#convert index to date format
pageview_df['date'] = pageview_df.index
pageview_df['date'] = pageview_df['date'].apply(lambda s: pd.to_datetime(s[:8], format='%Y%m%d'))
pageview_df = pageview_df.set_index('date')
#full data with all languages
pageview_df.head()

/var/folders/n2/lkxf5xgn5qd0l88pblwqxc6h0000gn/T/ipykernel_37948/3927525610.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pageview_df['date'] = pageview_df.index


,ar,en,es,id,ko,pt,zh,de,he,ru,...,io,li,mi,mzn,nds-nl,nrm,szy,tet,tt,wa
date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,108,4509,341,31,81,79,4821,2747,58,502,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-02,160,6780,477,46,177,86,11784,3699,40,511,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,90,6524,553,49,298,109,7161,1473,132,852,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-04,130,7220,506,145,215,101,7615,1131,102,1821,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-05,134,8016,464,112,192,93,8165,1077,89,1925,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:

compression_opts = dict(method='gzip', archive_name='page_views_covid_related.csv')  
pageview_df.to_csv('page_views_covid_related.csv.gz', index=True, compression= 'gzip')  


In [11]:
#Rules:
#language is spoken by at least 75% of the population of the country
#country's population speaking the language must represent at least 75% of the total population using the language
country_own_lang = {"Italy" : "it", "Russia": "ru", "China": "zh", "Albania": "sq", 
"Bangladesh": "bn", "Bostwana": "tn", "Cambogia": "km", "Croatia": "hr", "Greece": "el", "Sweden": "sv", "Finland": "fi", "Norway": "no",
 "Malaysian": "ms", "Israel": "he", "Lithuania": "lt", "Serbia": "sr", "Slovakia": "sk", "Slovenia": "sl", "Turkey": "tr",
 "Vietnam": "vi", "Bulgaria": "bg", "Czeck Republic": "cs", "Denmark": "da", "Georgia": "ka", "German": "de", 
 "Hungary": "hu", "Iceland": "is", "Japan": "ja", "Kazakhstan": "kk", "South Korea": "ko", "Kyrgyzstan": 'ky', "Netherland": "nl", "Poland": "pl", 
 "Romania": "ro", "Tajikistan": "tg", "Thailand": "th", "Azerbaijan": "az", "Mongolia": "mn"}
 

In [4]:
pageview_df = pd.read_csv("page_views_covid_related.csv.gz")

In [19]:
pageview_df_imp_country = pageview_df[["date"] + list(country_own_lang.values())].set_index('date')
pageview_df_imp_country.head(150)

,it,ru,zh,sq,bn,tn,km,hr,el,sv,...,kk,ko,ky,nl,pl,ro,tg,th,az,mn
date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,252,502,4821,0.0,NaN,NaN,NaN,10,14,56,...,0.0,81,NaN,131,357,17,NaN,4,0,NaN
2020-01-02,419,511,11784,NaN,NaN,NaN,NaN,25,11,56,...,0.0,177,NaN,137,225,24,NaN,21,2,NaN
2020-01-03,403,852,7161,0.0,NaN,NaN,NaN,19,14,67,...,2.0,298,NaN,139,258,32,NaN,19,0,NaN
2020-01-04,767,1821,7615,0.0,NaN,NaN,NaN,31,8,77,...,0.0,215,NaN,172,215,25,NaN,30,0,NaN
2020-01-05,1163,1925,8165,0.0,NaN,NaN,NaN,16,16,86,...,0.0,192,NaN,184,280,53,NaN,31,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-25,38490,60896,46784,249.0,2810.0,12.0,12.0,726,2651,2718,...,93.0,6095,11.0,9738,15275,3869,13.0,1928,222,30.0
2020-05-26,37156,58854,46835,426.0,3293.0,5.0,10.0,746,2656,2606,...,92.0,5973,14.0,8633,13422,3670,149.0,1895,243,6.0
2020-05-27,34292,56335,42890,362.0,2923.0,11.0,9.0,581,2753,2891,...,120.0,5884,22.0,8370,13602,3168,75.0,1692,259,9.0
